### 환경설정
- API Key를 환경변수로 관리하도록 설정
- Library 설치


In [ ]:
# 구글 드라이브를 코랩 환경에 마운트.
# 이를 통해 드라이브에 저장된 파일(.env 등)에 접근 가능.
from google.colab import drive
drive.mount('/content/drive')

# API 키 파일이 저장된 기본 경로를 설정.
base_path = '/content/drive/MyDrive/Colab Notebooks/AI/08_data_argument/'

# 폴더 생성 코드
# base_path 경로에 해당하는 폴더가 없으면 생성 (-p 옵션).
# makedirectory / 위치는 내 구글
!mkdir -p "{base_path}"

Mounted at /content/drive


In [ ]:
# Colab 환경에서 .env 파일을 생성하고 API 키를 저장하는 명령어.
# 실제 키를 {your_api_key} 부분에 입력
!echo "UPSTAGE_API_KEY={my_api}" > "/content/drive/MyDrive/Colab Notebooks/AI/08_data_argument/.env"

In [ ]:
# .env 파일에서 환경 변수를 로드하기 위한 라이브러리.
from dotenv import load_dotenv
# 운영체제의 환경 변수를 가져오기 위한 함수.
from os import getenv

# .env 파일을 로드하여 환경 변수를 설정.
load_dotenv(base_path + ".env")

# getenv 함수를 사용해 "UPSTAGE_API_KEY"라는 이름의 환경 변수 값을 가져옴.
UPSTAGE_API_KEY = getenv("UPSTAGE_API_KEY")

# API 키가 성공적으로 로드되었는지 확인하고 메시지를 출력.
if UPSTAGE_API_KEY:
    print("Success API Key Setting!")
else:
    print(f"ERROR: Failed to load UPSTAGE_API_KEY from {base_path}")


Success API Key Setting!


In [ ]:
!pip install openai

# upstage API 사용하기

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url= "https://api.upstage.ai/v1",
)

stream = client.chat.completions.create(
    # 내가 사용할 모델
    model = "solar-pro2",
    messages=[
        {
            "role":"user",
            "content": "ㅎㅇ 춥노",
        }
    ],
    # 응답을 한 번에 받지 않고 생성되는 대로 조각단위로 실시간 수신
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='')

# stream이 false인 경우
# print(stream.choices[0].message.content)

ㅎㅇ! 정말 춥네요~❄️  
오늘은 특히 바람이 세게 부는 것 같아요.  
따뜻하게 입고 다니시나요? ☕️  
(혹시 추워서 단거 당긴다? 핫초코 한 잔의 유혹을 느껴봐요...💕)  

> *추위에 맞서는 작은 팁*  
> - 목이 따뜻해야 체온 유지에 도움됨! 목도리 필수!  
> - 카페인에 약한 사람은 생강차 추천~ 몸이 따뜻해져요!  

감기 조심하시고, 오늘 하루 잘 버텨봐요! 💪🔥

# HTTPX
- python 용 HTTP 클라이언트
    - request도 httpx
- 비동기 통신을 지원
    - 여러 API 요청을 보낼 때 하나의 요청이 끝날 때 까지 기다리지 않고 동시에 병렬적으로 처리

In [ ]:
import httpx
import asyncio

async def call_chat_completion(url, headers, payload):
    print(' call_chat_completion 시작')
    async with httpx.AsyncClient(timeout=30.0) as client:
        # await : 뒤에 작성된 비동기 작업이 종료가 될 때까지 대기
        # 만약 다른 비동기 작업이 예정되어 있다면 해당 작업을 수행하기 시작
        response = await client.post(url, headers=headers, json=payload)
        # HTTP 오류가 발생하면 예외를 발생시키도록
        response.raise_for_status()
        data = response.json()

        return data['choices'][0]['message']['content']

In [ ]:
# 여러 비동기 작업을 실행하고 결과를 처리하는 메인 함수.
async def request(tasks):
    print('이제 각 요청 실행 시작')
    # asyncio.gather(*tasks): 리스트에 담긴 모든 비동기 작업을 동시에 실행하고,
    # 모든 작업이 완료될 때까지 기다린 후 결과를 리스트로 모아서 반환.
    results = await asyncio.gather(*tasks)
    print('모든 요청 완료')
    print()

    # 완료된 결과를 하나씩 출력.
    for i, res in enumerate(results, 1):
        print(f'Response {i}')
        print(res)
        print('=' * 20)


# API 엔드포인트 및 인증 헤더 설정.
url = "https://api.upstage.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {UPSTAGE_API_KEY}",
}

# 동시에 보낼 여러 개의 프롬프트를 리스트로 준비.
prompts = [
    "농담 하나만 해 줘",
    "프랑스의 수도는 어디야?",
]

# 실행할 비동기 작업들을 담을 리스트.
tasks = []
# 각 프롬프트에 대해 API 요청 페이로드를 생성.
for prompt in prompts:
    payload = {
        "model": "solar-pro2",
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        # stream=False: 전체 응답을 한 번에 받도록 설정.
        "stream": False,
    }
    # call_chat_completion 함수를 호출하여 비동기 작업(Task) 객체를 생성하고 리스트에 추가.
    # 'await'가 없으므로 함수가 바로 실행되지 않고, 실행 대기 상태의 객체만 만들어짐.
    print('태스크 생성')
    tasks.append(call_chat_completion(url, headers, payload))
    print('아직 upstage API 호출 전')

# 준비된 모든 태스크를 동시에 실행.
await request(tasks)

태스크 생성
아직 upstage API 호출 전
태스크 생성
아직 upstage API 호출 전
이제 각 요청 실행 시작
 call_chat_completion 시작
 call_chat_completion 시작
모든 요청 완료

Response 1
물론이죠! 여기 갑니다~  

**"사과가 왜 학교에 갔는지 알아?**  
**사과해서!!!"**  

(애플이 사과하는 게 아니라, '사과(謝過)하러' 갔대서... 😅)  

혹시 더 필요하신가요? 다른 버전으로 바꿔드릴게요~!
Response 2
프랑스의 수도는 **파리(Paris)**입니다. 파리는 정치, 경제, 문화의 중심지이며, 세계적으로 유명한 관광지(에펠탑, 루브르 박물관 등)와 예술, 역사가 풍부한 도시입니다.  

혹시 프랑스나 파리에 대해 더 궁금한 점이 있으면 언제든 물어보세요! 😊


### 응답을 JSON으로 고정

In [ ]:
import json

# LLM의 응답을 구조화된 JSON 형식으로 강제하기 위한 설정.
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "수도 정보",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "capital": {"type": "string"},
                "translation": {"type": "string", "description": "수도의 영어 번역"},
            },
            "required": ["capital", "translation"],
        },
    },
}

# client.chat.completions.create 메서드를 호출하여 LLM에 요청.
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "한국의 수도는 어디야?",
        }
    ],
    # 위에서 정의한 JSON 스키마를 적용하여 응답 형식을 강제.
    response_format=response_format,
)

# LLM의 응답은 JSON 형식의 '문자열'이므로,
# json.loads를 사용하여 파이썬 딕셔너리 객체로 변환.
structured_dictionary = json.loads(response.choices[0].message.content)

# 딕셔너리로 변환된 구조화된 응답을 출력.
print("Structured Response:")
for key, value in structured_dictionary.items():
    print(f"{key}: {value}")

Structured Response:
capital: 서울
translation: Seoul


# 데이터 합성

- LLM을 이용해 인공 데이터를 만드는 작업
    - 데이터 증강은 이미 있는 데이터를 변형시켜 양을 늘리는 작업
- 학습 데이터가 부족하거나 실제 데이터를 수집하기 어려울 때

## 기본 구조

효과적인 프롬프트를 작성하는 가이드라인

1. 역할(role): AI에게 특정 역할을 부여하고 답변의 전문성과 톤앤매너를 설정

ex. 영화광, 영화 평론가 .. 등

2. 목표(task): AI가 수행해야 할 작업을 구체적이고 명확하게 지시

3. 조건(constraints): 답변 형식, 스타일, 내용 등에 대한 제약 조건을 설정

## 데이터 합성 프롬프트의 핵심

1. 다양성: 모델의 무작위성을 조절하여 새롭고 다채로운 데이터 생성
- `temperature` : 확률분포의 모양을 조절
- `top_p` : 핵심 샘플링이라고도 부름. 후보 단어의 범위를 동적으로 조절
    - 적당성이 확보되는 수준으로 조절해야함
2. 일관성 : 생성된 데이터가 일관된 구조 (JSON 같이)를 가지도록 프롬프트에 명시

### 데이터 생성 해보기

In [ ]:
# Upstage API와 통신하기 위해 openai 라이브러리를 임포트합니다.
from openai import OpenAI
# LLM의 응답(JSON 형식의 문자열)을 파이썬 딕셔너리로 변환하기 위해 json 라이브러리를 임포트합니다.
import json

# 시스템 프롬프트: AI 모델의 역할과 기본 행동 지침을 정의합니다.
SYSTEM_PROMPT = """
당신은 세상의 모든 영화를 꿰뚫고 있는 영화 전문가 '시네마스터'입니다.
사용자의 요청에 맞춰 영화를 추천하는 역할을 맡고 있습니다. 영화는 반드시 하나만 추천합니다.
"""

# 추가 규칙 프롬프트: AI 모델의 말투나 답변 스타일 등 세부 규칙을 정의합니다.
RULE = """
친구가 소개 해주는 듯 부드럽고 친근한 말투로 답변합니다.
특히, recommended_reason 항목에서는 친구가 엄청 호들갑 떨듯이 설명해 주세요.
"""

# 응답 형식 프롬프트: LLM의 답변을 구조화된 JSON 형식으로 강제하기 위한 설정입니다.
response_format = {
    "type": "json_schema",      # 응답 형식을 JSON 스키마로 지정합니다.
    "json_schema": {
        "name": "영화 추천",      # 스키마의 이름을 지정합니다.
        "strict": True,          # 엄격 모드: 스키마에 맞지 않는 응답이 오면 오류를 발생시킵니다.
        "schema": {
            "type": "object",    # 응답의 최상위 타입이 객체(딕셔너리)임을 지정합니다.
            "properties": {      # 객체에 포함될 속성들을 정의합니다.
                "movie_name": {"type": "string"},
                "year": {"type": "integer"},
                "reason": {"type": "string"},
                "description": {"type": "string", "description": "영화에 대한 설명"},
                "recommended_reason": {"type": "string", "description": "이 영화를 추천하는 추가 이유"}
            },
            # 필수적으로 포함되어야 할 속성들을 지정합니다.
            "required": ["movie_name", "year", "reason", "description", "recommended_reason"]
        }
    }
}

# Upstage API 클라이언트를 생성합니다.
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

# client.chat.completions.create 메서드를 호출하여 LLM에 요청을 보냅니다.
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "system",
            "content": RULE
        },
        {
            "role": "user",
            "content": "공포 영화를 추천해줘"
        }
    ],
    response_format=response_format,  # 위에서 정의한 JSON 응답 형식을 적용합니다.
    temperature=0.5,                  # 다양성을 위해 temperature를 0.5로 설정합니다.
    max_tokens=1000,                  # 응답의 최대 길이를 1000 토큰으로 제한합니다.
    top_p=1.0,                        # top_p를 1.0으로 설정하여 모든 단어를 후보로 고려합니다.
    n=1,                              # 1개의 응답만 생성합니다.
    frequency_penalty=0.0,            # 특정 단어의 반복을 억제하지 않습니다.
    presence_penalty=0.0              # 새로운 주제의 등장을 장려하지 않습니다.
)

# 응답 결과에서 실제 텍스트 내용만 추출합니다.
output = response.choices[0].message.content
print(output)

{
  "movie_name": "컨저링",
  "year": 2013,
  "reason": "실화를 바탕으로 한 초자연적 공포가 압권인 작품",
  "description": "한 가족의 집에 깃든 악령을 퇴치하기 위한 에드 & 로레인 워렌 부부의 실화를 각색한 공포물. 실제 사건을 바탕으로 한 생생한 긴장감과 독특한 악령의 이미지가 관객을 압도합니다.",
  "recommended_reason": "와, 이건 진짜 친구한테 꼭 보라고 강요할 만한 영화야! 실제 사건을 바탕으로 해서 더 소름 돋고, 악령이 등장하는 장면들은 아직도 꿈에 나올 것 같아. 특히 '앤' 이라는 악령의 비주얼은 공포 영화 역사상 최고 수준이라니까? 소리 한 번 안 지르고 보는 게 불가능할 걸? 어둠 속에서 혼자 보는 건 절대 비추야 진짜."
}


## 생성 데이터 평가 (LLM as a Judge)

- 수 많은 합성 데이터를 사람이 직접 검수하는 것은 시간과 비용이 많이 들기 때문에 LLM을 평가자로 활용하는 기법
- 단순히 '좋다/나쁘다' 같은 정량적 평가를 넘어 '왜 그렇게 평가했는지'에 대한 이유도 생성하도록 하여 데이터 개선에 대한 피드백도 요구할 수 있음

### 생성 평가의 핵심
1. 평가 기준 설정 : 평가의 목적이 무엇인지 명확하게 설정 (지시한 Rule을 명확히 이행했는지)
2. 일관성 확보 (temperature=0) : 평가자는 창의적인 답변보다 일관되고 객관적인 평가를 내려야 신뢰가 가능  
3. 체계적인 평가 프롬프트 설계 : 평가자에게 필요한 모든 정보를 명확하게 제공

In [ ]:
# '평가자' 역할을 수행할 LLM에게 제공할 시스템 프롬프트입니다.
JUDGE_SYSTEM_PROMPT = """
당신의 역할은 모델 답변 자동 평가자입니다.

1. 입력 형식
    - 입력 프롬프트: [instruction]
    - 모델 답변: [output]
    - 평가 기준: [criteria]

2. 작업 지시
    - [instruction]에 따른 모델 결과물인 [output]을 평가합니다.
    - [output]은 [criteria]를 충족하는지 평가합니다.

3. 채점 원칙 (각 기준별 1–5점, 정수만)
    - 5점 (탁월): 기준을 완전히 충족. 오류·누락 없음. 구체적이고 실행가능.
    - 4점 (우수): 대체로 충족. 사소한 흠만 있음(정확성·구체성·형식 등에서 경미한 누락).
    - 3점 (보통): 핵심은 맞지만 눈에 띄는 약점 존재(누락, 모호함, 근거 부족 등).
    - 2점 (미흡): 중요한 요구를 여러 곳에서 놓침 또는 오류/비논리 다수.
    - 1점 (부적합): 전반적으로 요청과 어긋남, 의미있는 도움/근거 없음, 안전·정책 위반 가능성.

4. 출력 형식 (엄격 준수)
    - "score"는 1–5점의 정수로 평가한다.
    - "comment"는 한국어 1–3문장으로 평가한다. 구체적이고 실행 가능하게 작성한다.
    - 출력 형식은 JSON 형식인 response_format을 준수한다.
"""

# 평가자 LLM의 응답 형식을 JSON으로 강제하기 위한 설정입니다.
judge_response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "영화 추천 평가자",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "score": {"type": "integer"},
                "comment": {"type": "string", "description": "모델의 답변에 대한 평가 주석"}
            },
            "required": ["score", "comment"]
        }
    }
}

# 평가자 LLM에게 전달할 사용자 프롬프트 템플릿입니다.
USER_PROMPT = """
입력 프롬프트: {instruction}
모델 답변: {output}
평가 기준: {criteria}
"""

# 평가에 사용할 변수들을 정의합니다.
instruction = "공포 영화를 추천해줘"
# output 변수는 이전 데이터 생성 단계에서 얻은 결과물을 그대로 사용합니다.
# output = output

# 평가 기준은 생성 모델에게 전달했던 모든 지시사항(시스템 프롬프트, 규칙, 응답 형식)을 조합하여 만듭니다.
criteria = SYSTEM_PROMPT + RULE + str(response_format)

# 평가자 LLM에게 요청을 보냅니다.
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": JUDGE_SYSTEM_PROMPT
        },
        {
            "role": "user",
            # .format()을 사용해 템플릿에 실제 변수 값들을 채워 넣습니다.
            "content": USER_PROMPT.format(instruction=instruction, output=output, criteria=criteria)
        }
    ],
    temperature=0,  # 평가의 일관성을 위해 temperature를 0으로 설정합니다.
    response_format=judge_response_format
)

# 평가 결과를 JSON 문자열에서 파이썬 딕셔너리로 변환합니다.
judge_output = json.loads(response.choices[0].message.content)

# 최종 평가 결과를 확인합니다.
print(judge_output)

{'score': 5, 'comment': "모든 평가 기준을 탁월하게 충족했습니다. '컨저링'을 단일 추천하며, 전문가다운 정확한 정보(연도, 실화 기반 설명)와 친근한 말투("}
